In [ ]:
#traffic classifer model creation
import os
import random
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split

# Config
IMAGES_PATH = "../output/intersectionlight_images_merged.npy"
LABELS_PATH = "../output/intersectionlight_labels_merged.npy"
SAVE_DIR = "./checkpoints"
EPOCHS = 500
BATCH_SIZE = 32
LR = 1e-7
VAL_SPLIT = 0.2
RESIZE = 224
SEED = 42
USE_PRETRAINED = True
MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)

# Seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Dataset loader for .npy files
class NpyImageDataset(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.images = np.load(images_path, mmap_mode='r')
        self.labels = np.load(labels_path, mmap_mode='r')
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        arr = self.images[idx]
        if arr.dtype != np.uint8:
            arr = np.clip(arr, 0, 255).astype(np.uint8)
        img = transforms.functional.to_pil_image(arr)
        x = self.transform(img) if self.transform else transforms.ToTensor()(img)
        return x, int(self.labels[idx])

# Build ResNet18
def build_model(num_classes=2, pretrained=True):
    try:
        weights = models.ResNet18_Weights.DEFAULT if pretrained else None
        model = models.resnet18(weights=weights)
    except:
        model = models.resnet18(pretrained=pretrained)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Evaluate helper
def evaluate(model, loader, device):
    model.eval()
    losses, correct, total = [], 0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            losses.append(F.cross_entropy(logits, yb).item())
            correct += (logits.argmax(1) == yb).sum().item()
            total += yb.size(0)
    return float(np.mean(losses)), correct / max(total, 1)

# Training loop
def main():
    set_seed(SEED)
    if not Path(IMAGES_PATH).exists() or not Path(LABELS_PATH).exists():
        raise FileNotFoundError("Check IMAGES_PATH and LABELS_PATH")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_tfms = transforms.Compose([
        transforms.Resize((RESIZE, RESIZE)),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.02),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ])
    val_tfms = transforms.Compose([
        transforms.Resize((RESIZE, RESIZE)),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ])

    # Split data
    full_ds = NpyImageDataset(IMAGES_PATH, LABELS_PATH)
    labels = np.array([int(x) for x in full_ds.labels])
    idx_train, idx_val = train_test_split(
        np.arange(len(full_ds)), test_size=VAL_SPLIT, stratify=labels, random_state=SEED
    )
    train_ds = Subset(NpyImageDataset(IMAGES_PATH, LABELS_PATH, transform=train_tfms), idx_train)
    val_ds = Subset(NpyImageDataset(IMAGES_PATH, LABELS_PATH, transform=val_tfms), idx_val)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = build_model(pretrained=USE_PRETRAINED).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))

    best_acc = 0
    os.makedirs(SAVE_DIR, exist_ok=True)
    best_path = os.path.join(SAVE_DIR, 'traffic_lights_model.pt')

    for epoch in range(1, EPOCHS+1):
        model.train()
        losses, correct, total = [], 0, 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
                logits = model(xb)
                loss = F.cross_entropy(logits, yb)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            losses.append(loss.item())
            correct += (logits.argmax(1) == yb).sum().item()
            total += yb.size(0)
        val_loss, val_acc = evaluate(model, val_loader, device)
        print(f"Epoch {epoch:02d} | Train Acc {correct/max(total,1):.4f} | Val Acc {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save({'model_state_dict': model.state_dict()}, best_path)
            print(f"  Saved new best model (val_acc={best_acc:.4f})")

    print(f"Training done. Best val_acc={best_acc:.4f}")

# Quick prediction helper
def predict(image_array, weights_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = build_model(pretrained=False).to(device)
    model.load_state_dict(torch.load(weights_path, map_location=device)['model_state_dict'])
    model.eval()
    if image_array.dtype != np.uint8:
        image_array = np.clip(image_array, 0, 255).astype(np.uint8)
    img = transforms.functional.to_pil_image(image_array)
    tfm = transforms.Compose([
        transforms.Resize((RESIZE, RESIZE)),
        transforms.ToTensor(),
        transforms.Normalize(MEAN, STD),
    ])
    x = tfm(img).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = int(model(x).argmax(1).item())
    return pred

if __name__ == "__main__":
    main()


C:\Users\Pravin\AppData\Local\Temp\ipykernel_7512\2247371942.py:112: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda'))
C:\Users\Pravin\AppData\Local\Temp\ipykernel_7512\2247371942.py:124: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):


Epoch 01 | Train Acc 0.5250 | Val Acc 0.5000
  Saved new best model (val_acc=0.5000)
Epoch 02 | Train Acc 0.5250 | Val Acc 0.5000
Epoch 03 | Train Acc 0.5625 | Val Acc 0.5000
Epoch 04 | Train Acc 0.5500 | Val Acc 0.5000
Epoch 05 | Train Acc 0.5625 | Val Acc 0.5500
  Saved new best model (val_acc=0.5500)
Epoch 06 | Train Acc 0.5500 | Val Acc 0.5500
Epoch 07 | Train Acc 0.5625 | Val Acc 0.5500
Epoch 08 | Train Acc 0.4875 | Val Acc 0.5500
Epoch 09 | Train Acc 0.5500 | Val Acc 0.5500
Epoch 10 | Train Acc 0.5750 | Val Acc 0.5500
Epoch 11 | Train Acc 0.5375 | Val Acc 0.5500
Epoch 12 | Train Acc 0.5125 | Val Acc 0.5500
Epoch 13 | Train Acc 0.5500 | Val Acc 0.5500
Epoch 14 | Train Acc 0.5500 | Val Acc 0.5500
Epoch 15 | Train Acc 0.5125 | Val Acc 0.6000
  Saved new best model (val_acc=0.6000)
Epoch 16 | Train Acc 0.5375 | Val Acc 0.6000
Epoch 17 | Train Acc 0.5875 | Val Acc 0.6000
Epoch 18 | Train Acc 0.5625 | Val Acc 0.6000
Epoch 19 | Train Acc 0.5375 | Val Acc 0.6500
  Saved new best model (v